In [124]:
%matplotlib inline
import numpy as np
from pulp import *
import pandas as pd
import pprint
import csv
MEPS=1.03-6

In [125]:
df=pd.read_csv('premiere_salary.csv',header=0,usecols=[0])
df

,Name
0,Kevin De Bruyne
1,Raheem Sterling
2,Paul Pogba
3,Mesut Ozil
4,Anthony Martial
5,Sergio Aguero
6,Mohamed Salah
7,Marcus Rashford
8,Harry Kane
9,Tanguy Ndombele


In [126]:
DMUs=df.values
print(DMUs)

[['Kevin De Bruyne']
 ['Raheem Sterling']
 ['Paul Pogba']
 ['Mesut Ozil']
 ['Anthony Martial']
 ['Sergio Aguero']
 ['Mohamed Salah']
 ['Marcus Rashford']
 ['Harry Kane']
 ['Tanguy Ndombele']]


In [127]:
dfy=pd.read_csv('premiere_salary.csv',header=0,index_col=0,usecols=[0,2,3,4,5,6])
dfy

,Appearance,Goals,Assists,Shots on Target,Passes per game
Name,,,,,
Kevin De Bruyne,33,11,19,28,54
Raheem Sterling,31,17,2,32,36
Paul Pogba,14,1,4,8,65
Mesut Ozil,18,1,3,3,47
Anthony Martial,30,17,6,38,23
Sergio Aguero,24,16,4,27,15
Mohamed Salah,32,19,10,58,29
Marcus Rashford,29,17,8,42,31
Harry Kane,28,17,3,36,20


In [128]:
y=dfy

In [129]:
print(dfy)

                 Appearance  Goals  Assists  Shots on Target  Passes per game
Name                                                                         
Kevin De Bruyne          33     11       19               28               54
Raheem Sterling          31     17        2               32               36
Paul Pogba               14      1        4                8               65
Mesut Ozil               18      1        3                3               47
Anthony Martial          30     17        6               38               23
Sergio Aguero            24     16        4               27               15
Mohamed Salah            32     19       10               58               29
Marcus Rashford          29     17        8               42               31
Harry Kane               28     17        3               36               20
Tanguy Ndombele          21      2        3                4               31


In [130]:
y.shape

(10, 5)

In [131]:
dfx=pd.read_csv('premiere_salary.csv',header=0,usecols=[1])
dfx

,Salary (€)
0,16683333
1,15600000
2,15080000
3,13975000
4,13000000
5,11967000
6,10400000
7,10400000
8,10400000
9,10400000


In [132]:
x=dfx

In [133]:
x.shape

(10, 1)

In [134]:
n,m=y.shape
n,s=x.shape

In [135]:
res=[]

In [136]:
for o in range(n):
    prob=LpProblem('DMU_'+str(o),LpMaximize)
    #　重み
    v=[LpVariable('v'+str(i),lowBound=0,
                  cat='Continuous') for i in range(m)]
    u=[LpVariable('u'+str(i),lowBound=0,
                  cat='Continuous') for i in range(s)]
    #　目的関数
    prob += lpDot(u,y.iloc[o,:])

    # 制約条件
    prob += lpDot(v,x.iloc[o,:])==1,'Normalize'+str(o)

    for j in range(n):
        prob += lpDot(u,y.iloc[j,:]) <= lpDot(v,x.iloc[j,:])

    prob.solve()
    vs=np.array([v[i].varValue for i in range(m)]) #v*
    us=np.array([u[i].varValue for i in range(s)]) #u*
    print('DMU_'+str(o),': ',DMUs[o])
    print('vs: ',vs)
    print('us: ',us)
    print(' ')
#     # 参照集合作成
#     (eo,)=np.where(np.abs(np.dot(vs,x)-np.dot(us,y))<= MEPS)

#     res.append([DMUs[o],value(prob.objective),
#                 set(eo),tuple(vs),tuple(us)])

    res.append([DMUs[o],value(prob.objective)])

DMU_0 :  ['Kevin De Bruyne']
vs:  [5.9940061e-08 None None None None]
us:  [0.01948052]
 
DMU_1 :  ['Raheem Sterling']
vs:  [6.4102564e-08 None None None None]
us:  [0.02083333]
 
DMU_2 :  ['Paul Pogba']
vs:  [6.6312997e-08 None None None None]
us:  [0.02155172]
 
DMU_3 :  ['Mesut Ozil']
vs:  [7.1556351e-08 None None None None]
us:  [0.02325581]
 
DMU_4 :  ['Anthony Martial']
vs:  [7.6923077e-08 None None None None]
us:  [0.025]
 
DMU_5 :  ['Sergio Aguero']
vs:  [8.3563132e-08 None None None None]
us:  [0.02715802]
 
DMU_6 :  ['Mohamed Salah']
vs:  [9.6153846e-08 None None None None]
us:  [0.03125]
 
DMU_7 :  ['Marcus Rashford']
vs:  [9.6153846e-08 None None None None]
us:  [0.03125]
 
DMU_8 :  ['Harry Kane']
vs:  [9.6153846e-08 None None None None]
us:  [0.03125]
 
DMU_9 :  ['Tanguy Ndombele']
vs:  [9.6153846e-08 None None None None]
us:  [0.03125]
 


In [137]:
print(res)

[[array(['Kevin De Bruyne'], dtype=object), 0.64285716], [array(['Raheem Sterling'], dtype=object), 0.645833323], [array(['Paul Pogba'], dtype=object), 0.30172413600000003], [array(['Mesut Ozil'], dtype=object), 0.41860465199999997], [array(['Anthony Martial'], dtype=object), 0.75], [array(['Sergio Aguero'], dtype=object), 0.6517924319999999], [array(['Mohamed Salah'], dtype=object), 1.0], [array(['Marcus Rashford'], dtype=object), 0.90625], [array(['Harry Kane'], dtype=object), 0.875], [array(['Tanguy Ndombele'], dtype=object), 0.65625]]


In [138]:
df_f=pd.DataFrame(res,columns=['Name','Efficiency'])
df_s=df_f.sort_values('Efficiency',ascending=False)
df_s

,Name,Efficiency
6,[Mohamed Salah],1.000000
7,[Marcus Rashford],0.906250
8,[Harry Kane],0.875000
4,[Anthony Martial],0.750000
9,[Tanguy Ndombele],0.656250
5,[Sergio Aguero],0.651792
1,[Raheem Sterling],0.645833
0,[Kevin De Bruyne],0.642857
3,[Mesut Ozil],0.418605
2,[Paul Pogba],0.301724


In [114]:
# def DEA_CCR(x,y,DMUs):
#     n,m=x.shape
#     n,s=y.shape
    
#     res=[]
    
#     for o in range(n):
#         prob=LpProblem('DMU_'+str(o),LpMaximize)
#         #　重み
#         v=[LpVariable('v'+str(i),lowBound=0,
#                       cat='Continuous') for i in range(m)]
#         u=[LpVariable('u'+str(i),lowBound=0,
#                       cat='Continuous') for i in range(s)]
#         #　目的関数
#         prob += lpDot(u,y.iloc[:,o])

#         # 制約条件
#         prob += lpDot(v,x.iloc[:,o])==1,'Normalize'+str(o)

#         for j in range(n):
#             prob += lpDot(u,y.iloc[:,j]) <= lpDot(v,x.iloc[:,j])

#         prob.solve()
#         vs=np.array([v[i].varValue for i in range(m)]) #v*
#         us=np.array([v[i].varValue for i in range(m)]) #u*
#         # 参照集合作成
#         (eo,)=np.where(np.abs(np.dot(vs,x)-np.dot(us,y))<= MEPS)
#         res.append([DMUs[o],value(prob.objective),
#                     set(eo),tuple(vs),tuple(us)])
#     return res

TypeError: unsupported operand type(s) for ^: 'float' and 'float'